In [2]:
from __future__ import absolute_import, division, print_function
import os 



In [3]:
# Data preprocessing  
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size=16

train_aug = ImageDataGenerator(rescale=1.0/255)
                               

test_aug = ImageDataGenerator(rescale=1.0/255)

train_generator = train_aug.flow_from_directory('GoT-images/train', # Path to directory with training images
                                                target_size=(224,224),
                                                batch_size=batch_size,
                                                shuffle=True,
                                                class_mode='categorical')

validation_generator = test_aug.flow_from_directory('GoT-images/valid', # Path to directory with validation images 
                                                   target_size=(224,224),
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   class_mode='categorical')

Found 37661 images belonging to 2 classes.
Found 7617 images belonging to 2 classes.


In [6]:
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization 

# Version 1.0

model = Sequential()

model.add(Conv2D(filters=24, kernel_size=6, strides=1, padding='same', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))


model.add(Conv2D(filters=48, kernel_size=5, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=4, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))

model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(200))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7)) 


model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 24)      2616      
_________________________________________________________________
batch_normalization_6 (Batch (None, 224, 224, 24)      96        
_________________________________________________________________
activation_6 (Activation)    (None, 224, 224, 24)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 24)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 112, 112, 24)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 48)        28848     
_________________________________________________________________
batch_normalization_7 (Batch (None, 56, 56, 48)        192       
__________

In [7]:
from keras.optimizers import Adam

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


epochs = 50

# number of training images
nb_training_size = 37661.0   

# number of validation images
nb_validation_size = 7617.0

# saves best model to filepath
checkpointer = ModelCheckpoint(filepath='saved_models/ver1_0.hdf5', 
                               verbose=1, save_best_only=True)

# stops training if validation loss stops improving
early_stop = EarlyStopping(monitor='val_loss', patience=5)


model.fit_generator(train_generator,
                   steps_per_epoch=nb_training_size//batch_size,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=nb_validation_size//batch_size,
                   verbose=1,
                   callbacks=[checkpointer, early_stop])

Epoch 1/50
2353/2353 [==============================] - 937s - loss: 0.7088 - acc: 0.5937 - val_loss: 0.9077 - val_acc: 0.5110
Epoch 2/50
2353/2353 [==============================] - 885s - loss: 0.6012 - acc: 0.6824 - val_loss: 0.7996 - val_acc: 0.5610
Epoch 3/50
2353/2353 [==============================] - 891s - loss: 0.5799 - acc: 0.7009 - val_loss: 0.8782 - val_acc: 0.5456
Epoch 4/50
2353/2353 [==============================] - 887s - loss: 0.5600 - acc: 0.7176 - val_loss: 0.5030 - val_acc: 0.7621
Epoch 5/50
2353/2353 [==============================] - 888s - loss: 0.5465 - acc: 0.7286 - val_loss: 0.5687 - val_acc: 0.7021
Epoch 6/50
2353/2353 [==============================] - 891s - loss: 0.5317 - acc: 0.7353 - val_loss: 0.4685 - val_acc: 0.7798
Epoch 7/50
2353/2353 [==============================] - 895s - loss: 0.5257 - acc: 0.7401 - val_loss: 0.6519 - val_acc: 0.6723
Epoch 8/50
2353/2353 [==============================] - 892s - loss: 0.5179 - acc: 0.7477 - val_loss: 0.5552 - 